# **WFLOW: PRODUCTS RELATED TO DIGITAL ELEVATION MODEL**

## *PACKAGES*


In [1]:
include(raw"d:\JOE\MAIN\MODELS\WFLOW\WflowDataJoe\WflowRaster.jl\src\GeoRaster.jl")
include(raw"d:\JOE\MAIN\MODELS\WFLOW\WflowDataJoe\WflowRaster.jl\src\Parameters.jl")

using Rasters, Shapefile, GeoTIFF, ArchGDAL, GeoFormatTypes, GeoArrays, Extents
using Geomorphometry
# using CairoMakie, GLMakie

# CairoMakie.activate!()


## **DEM OFFICIAL GRID SIZE**

	https://rafaqz.github.io/Rasters.jl/dev/api#Rasters.resample-Tuple
method: A Symbol or String specifying the method to use for resampling. From the docs for gdalwarp:

**:near:** nearest neighbour resampling (default, fastest algorithm, worst interpolation quality).
:bilinear: bilinear resampling.
:cubic: cubic resampling.
:cubicspline: cubic spline resampling.
:lanczos: Lanczos windowed sinc resampling.
:average: average resampling, computes the weighted average of all non-NODATA contributing pixels. rms root mean square / quadratic mean of all non-NODATA contributing pixels (GDAL >= 3.3)
:mode: mode resampling, selects the value which appears most often of all the sampled points.
:max: maximum resampling, selects the maximum value from all non-NODATA contributing pixels.
:min: minimum resampling, selects the minimum value from all non-NODATA contributing pixels.
:med: median resampling, selects the median value of all non-NODATA contributing pixels.
:q1: first quartile resampling, selects the first quartile value of all non-NODATA contributing pixels.
:q3: third quartile resampling, selects the third quartile value of all non-NODATA contributing pixels.
:sum: compute the weighted sum of all non-NODATA contributing pixels (since GDAL 3.1)

## **DEM OPERATIONS**

In [2]:

# Reading data
   Path_InputGis_Dem    = joinpath(Path_Root, Path_InputGis, Dem_Input)
   Dem_Raw              = Rasters.Raster(Path_InputGis_Dem)

   # Resample Dem in 2 steps
	Crs_GeoFormat = GeoFormatTypes.convert(WellKnownText, EPSG(Crs))

	Dem_Resample₀ = Rasters.replace_missing(Dem_Raw, missingval=NaN)

	Dem_Resample₁ = Rasters.resample(Dem_Resample₀; res=ΔX₁, method=ResampleMethod₁, crs=Crs_GeoFormat)

	Dem_Resample = Rasters.resample(Dem_Resample₁; res=ΔX₂, method=ResampleMethod₂, crs=Crs_GeoFormat)

   # Writting the output
   Path_OutputJulia_Dem = joinpath(Path_Root, Path_OutputJulia, Dem_Julia)
   Rasters.write(Path_OutputJulia_Dem, Dem_Resample; ext=".tiff", missingval= NaN, force=true, verbose=true)

# Getting the metadata
	Metadatas = geoRaster.RASTER_METADATA(Path_OutputJulia_Dem; Verbose=false);

# # Dem_Resample = Rasters.aggregate(locus=Rasters.Center(), Dem_Raw, (Y(ΔX), X(ΔX)); skipmissingval=true)



Main.geoRaster.METADATA(921, 571, 5, 5, 143100.001, 147700.001, 42625.008, 39775.008, 29902, WellKnownText{GeoFormatTypes.CRS}(GeoFormatTypes.CRS(), "PROJCS[\"TM65 / Irish Grid\",GEOGCS[\"TM65\",DATUM[\"TM65\",SPHEROID[\"Airy Modified 1849\",6377340.189,299.3249646,AUTHORITY[\"EPSG\",\"7002\"]],AUTHORITY[\"EPSG\",\"6299\"]],PRIMEM[\"Greenwich\",0,AUTHORITY[\"EPSG\",\"8901\"]],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]],AUTHORITY[\"EPSG\",\"4299\"]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"latitude_of_origin\",53.5],PARAMETER[\"central_meridian\",-8],PARAMETER[\"scale_factor\",1.000035],PARAMETER[\"false_easting\",200000],PARAMETER[\"false_northing\",250000],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"Easting\",EAST],AXIS[\"Northing\",NORTH],AUTHORITY[\"EPSG\",\"29902\"]]"), 1, Extent(X = (143100.001, 147700.001), Y = (39775.008, 42625.008)))

In [3]:
using CairoMakie, GLMakie
if Flag_Plots
   Makie.plot(hillshade(Dem_Resample))
end


## ====== **PLEASE RUN PYTHON CODE** =======

++===========================================++


## **CATCHMENT:** *Reading catchment boundary from Python used as a mask*

In [4]:
Path_InputPython_Subcatchment = joinpath(Path_Root, Path_OutputPython, Subcatch_Python)
Subcatchment₀ = Rasters.Raster(Path_InputPython_Subcatchment)

# Assuring that it has the same grid size as the Dem_Resample
Subcatchment = Rasters.resample(Subcatchment₀; to=Dem_Resample, missingval= NaN)

# Writting the output
Path_OutputWflow_Subcatchment = joinpath(Path_Root, Path_OutputWflow, Subcatch_Wflow)
Rasters.write(Path_OutputWflow_Subcatchment, Subcatchment; ext=".tiff", missingval= NaN, force=true, verbose=true)


"d:\\JOE\\MAIN\\MODELS\\WFLOW\\DATA\\Timoleague\\OutputWflow\\wflow_subcatch.tiff"

In [5]:
if Flag_Plots
   Makie.plot(Subcatchment)
end


## **DEM:** *Mask to catchment area*

In [6]:
Dem_Resample_Mask = Rasters.mask(Dem_Resample; with=Subcatchment, missingval=NaN)

# Writting the output
Path_OutputJulia_DemMask = joinpath(Path_Root, Path_OutputJulia, Dem_Julia_Mask)
Rasters.write(Path_OutputJulia_DemMask, Dem_Resample_Mask; ext=".tiff", missingval= NaN, force=true, verbose=true)

# Getting the metadata
	Metadatas = geoRaster.RASTER_METADATA(Path_OutputJulia_DemMask; Verbose=true);

if Flag_Plots
   Makie.plot(hillshade(Dem_Resample_Mask))
end


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\OutputJulia\Timoleague_DEM_Mask.tif
Bands = 1
Crs = 29902
ΔX = 5.0
ΔY = 5.0
N_Width  = 921
N_Height = 571
Coord_X_Left = 143100.001, Coord_X_Right = 147700.001
Coord_Y_Top = 42625.008, Coord_Y_Bottom = 39775.008


## **LDD:** *Mask to catchment area* 

In [7]:
Path_InputPython_Ldd = joinpath(Path_Root, Path_OutputPython, Ldd_Python)
Ldd₀ = Rasters.Raster(Path_InputPython_Ldd)

# Assuring that it has the same grid size as the Dem_Resample
Ldd₁ = Rasters.resample(Ldd₀; to=Dem_Resample_Mask)

Ldd_Mask = Rasters.mask(Ldd₁; with=Subcatchment, missingval=0)

# Writting the output
Path_OutputWflow_LddMask = joinpath(Path_Root, Path_OutputWflow, Ldd_Wflow)
println(Path_OutputWflow_LddMask)
Rasters.write(Path_OutputWflow_LddMask, Ldd_Mask; ext=".tiff", missingval=NaN, force=true, verbose=true)

if Flag_Plots
   Makie.plot(Ldd_Mask)
end


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\OutputWflow\wflow_ldd.tiff


## **OUTLETS STATION**

In [8]:

Path_InputGis_Outlet  = joinpath(Path_Root, Path_InputGis, Outlet_Input)
println(Path_InputGis_Outlet)

# Load the shapefile
Outlet_Raw = Shapefile.Handle(Path_InputGis_Outlet)

Outlet = Rasters.rasterize(last, Outlet_Raw; res=ΔX₂, fill=1, missingval=NaN, crs=Metadatas.Crs_GeoFormat, to=Dem_Resample_Mask, shape=:point ,progress=true)

#TODO: check if the outlet is inside the subcatchment
# Outlet_Mask = Rasters.mask(Outlet; with=Subcatchment, missingval=NaN)

Path_OutputJulia_Outlet = joinpath(Path_Root, Path_OutputJulia, Outlet_Julia)
Rasters.write(Path_OutputJulia_Outlet, Outlet; ext=".tiff", force=true, verbose=true, missingval= NaN)

if Flag_Plots
   CairoMakie.heatmap(Outlet)
end


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\InputGis\Timoleague_Outlet_Hydro.shp


## **RIVER:** *Mask to catchment area*

In [9]:

Path_InputGis_River  = joinpath(Path_Root, Path_InputGis, River_Input)
println(Path_InputGis_River)

# Load the shapefile
River_Raw = Shapefile.Handle(Path_InputGis_River)

River = Rasters.rasterize(last, River_Raw; res=ΔX₂, fill=1.0, missingval=NaN, crs=Metadatas.Crs_GeoFormat, to=Dem_Resample_Mask, shape=:line ,progress=true)

River_Mask = Rasters.mask(River; with=Subcatchment, missingval=0)

Path_OutputWflow_River = joinpath(Path_Root, Path_OutputWflow, River_Wflow)
Rasters.write(Path_OutputWflow_River, River_Mask; ext=".tiff", force=true, verbose=true, missingval= 0)

if Flag_Plots
   CairoMakie.heatmap(River_Mask ; colormap=:romaO, colorrange=(0, 1))
end


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\InputGis\Timoleague_River.shp


Rasterizing... 100%|██████████████████████████████████████████████████| Time: 0:00:01


## **SLOPE:** *Reading Slope from Python & Mask*

In [10]:

Path_InputPython_Slope = joinpath(Path_Root, Path_OutputPython, Slope_Python)

Slope₀ = Rasters.Raster(Path_InputPython_Slope)

Slope_Mask₀ = Rasters.mask(Slope₀; with=Subcatchment, missingval=NaN);

Slope_Mask = Rasters.resample(Slope_Mask₀; to=Dem_Resample_Mask)

# Writting the output
Path_OutputWflow_Slope = joinpath(Path_Root, Path_OutputWflow, RiverSlope_Wflow)
Rasters.write(Path_OutputWflow_Slope, Slope_Mask; ext=".tiff", missingval= NaN, force=true, verbose=true)
println(Path_OutputWflow_Slope)

if Flag_Plots
   Makie.plot(Slope_Mask; colormap=:matter)
end


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\OutputWflow\RiverSlope.tiff


## **Rivers slope**

In [11]:
RiverSlope = Rasters.mask(Slope_Mask; with=River_Mask, missingval=NaN)

# RiverSlope = Float64.(RiverSlope)

Path_OutputWflow_RiverSlope = joinpath(Path_Root, Path_OutputWflow, RiverSlope_Wflow)
Rasters.write(Path_OutputWflow_RiverSlope, RiverSlope; ext=".tiff", force=true, verbose=true, missingval=0)
println(Path_OutputWflow_RiverSlope)

if Flag_Plots
   CairoMakie.heatmap(RiverSlope ; colormap=:romaO, colorrange=(0, 60))
end


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\OutputWflow\RiverSlope.tiff


## **River width**

In [12]:

RiverWidth = Float64.(River_Mask) .* P_RiverWidth

Path_OutputWflow_RiverWidth = joinpath(Path_Root, Path_OutputWflow, RiverWidth_Wflow)

Rasters.write(Path_OutputWflow_RiverWidth, RiverWidth; ext=".tiff", force=true, verbose=true, missingval=NaN)

println(Path_OutputWflow_RiverWidth)

if Flag_Plots
   CairoMakie.heatmap(RiverWidth; colormap=:romaO, colorrange=(0, 60))
end


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\OutputWflow\wflow_riverwidth.tiff


## **River depth**

In [13]:
RiverDepth = River_Mask .* P_RiverDepth

Path_OutputWflow_RiverDepth = joinpath(Path_Root, Path_OutputWflow, RiverDepth_Wflow)
println(Path_OutputWflow_RiverDepth)

Rasters.write(Path_OutputWflow_RiverDepth, RiverDepth; ext=".tiff", force=true, verbose=true, missingval=0)

if Flag_Plots
   CairoMakie.heatmap(RiverDepth; colormap=:romaO, colorrange=(0, 60))
end


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\OutputWflow\wflow_riverdepth.tiff


## **NETCDF**

In [14]:
include(raw"d:\JOE\MAIN\MODELS\WFLOW\WflowDataJoe\WflowRaster.jl\src\GeoRaster.jl")
include(raw"d:\JOE\MAIN\MODELS\WFLOW\WflowDataJoe\WflowRaster.jl\src\Parameters.jl")

NetCDF, Path_NetCDF_Full = geoRaster.TIFF_2_NETCDF(Ldd_Mask, Metadatas, River_Mask, River_Wflow, RiverDepth, RiverDepth_Wflow, RiverSlope, RiverSlope_Wflow, RiverWidth, RiverWidth_Wflow, Slope_Mask, Subcatch_Wflow, Subcatchment);


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\OutputNetCDF\instates-Timoleague.nc
wflow_ldd
wflow_subcatch
Slope
wflow_river
RiverSlope
wflow_riverwidth
wflow_riverdepth


In [15]:
 using NCDatasets
NetCdf = NCDataset(Path_NetCDF_Full)


Dataset: d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\OutputNetCDF\instates-Timoleague.nc
Group: /

Dimensions
   x = 921
   y = 571

Variables
  wflow_ldd   (921 × 571)
    Datatype:    Int64 (Int64)
    Dimensions:  x × y
    Attributes:
     units                = 1-9
     comments             = Derived from hydromt.flw.d8_from_dem

  wflow_subcatch   (921 × 571)
    Datatype:    Int64 (Int64)
    Dimensions:  x × y
    Attributes:
     units                = 1
     comments             = Derived from hydromt

  Slope   (921 × 571)
    Datatype:    Float64 (Float64)
    Dimensions:  x × y
    Attributes:
     units                = -
     comments             = Derived from hydromt

  wflow_river   (921 × 571)
    Datatype:    Int64 (Int64)
    Dimensions:  x × y
    Attributes:
     units                = 0/1
     comments             = Derived from hydromt

  RiverSlope   (921 × 571)
    Datatype:    Float64 (Float64)
    Dimensions:  x × y
    Attributes:
     units                = 

In [16]:
close(NetCdf)


closed Dataset

In [23]:
include(raw"d:\JOE\MAIN\MODELS\WFLOW\WflowDataJoe\WflowRaster.jl\src\GeoRaster.jl")
include(raw"d:\JOE\MAIN\MODELS\WFLOW\WflowDataJoe\WflowRaster.jl\src\Parameters.jl")

NetCDFmeteo, Path_NetCDFmeteo_Output, Time = geoRaster.TIMESERIES_2_NETCDF(Metadatas, Subcatchment);


d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\InputTimeSeries/TimeSeries_Process\forcing.Timoleague.csv
(921, 571)
(921, 571, 100)


Base.IOError: IOError: unlink("d:\\JOE\\MAIN\\MODELS\\WFLOW\\DATA\\Timoleague\\InputTimeSeries/TimeSeries_Wflow\\forcing-Timoleague.nc"): resource busy or locked (EBUSY)

In [18]:
 using NCDatasets
 Output_NCDatasets = NCDataset(Path_NetCDFmeteo_Output)


Dataset: d:\JOE\MAIN\MODELS\WFLOW\DATA\Timoleague\InputTimeSeries/TimeSeries_Wflow\forcing-Timoleague.nc
Group: /

Dimensions
   x = 921
   y = 571
   time = 1000

Variables
  time   (1000)
    Datatype:    Dates.DateTime (Float64)
    Dimensions:  time
    Attributes:
     units                = days since 1900-01-01 00:00:00
     calendar             = proleptic_gregorian

  precip   (921 × 571 × 1000)
    Datatype:    Float64 (Float64)
    Dimensions:  x × y × time
    Attributes:
     units                = mm
     comments             = precipitation

  pet   (921 × 571 × 1000)
    Datatype:    Float64 (Float64)
    Dimensions:  x × y × time
    Attributes:
     units                = mm
     comments             = potential evapotranspiration

  temp   (921 × 571 × 1000)
    Datatype:    Float64 (Float64)
    Dimensions:  x × y × time
    Attributes:
     units                = mm
     comments             = potential evapotranspiration

Global attributes
  title                =

In [19]:
close(NetCDFmeteo)


closed Dataset

In [20]:
# ================================================================
					# Plotting parameters
					ColourOption_No    = 1
					Linewidth          = 2
					height             = 400
					labelsize          = 20
					textcolor          = :blue
					textsize           = 20
					titlecolor         = :navyblue
					titlesize          = 18.0
					width              = height * 1.0
					xgridstyle         = :dash
					xgridvisible       = true
					xlabelSize         = 20
					xlabelpadding      = 5
					xminortickalign    = 1.0
					xminorticksvisible = true
					xtickalign         = 0.9 # 0 is inside and 1 is outside
					xticklabelrotation = π / 4.0
					xticksize          = 10
					xticksmirrored     = false
					xtickwidt          = 0.5
					xtrimspine         = false
					ygridstyle         = :dash
					ygridvisible       = false
					ylabelpadding      = xlabelpadding
					ylabelsize         = xlabelSize
					yminortickalign    = xminortickalign
					yminorticksvisible = true
					ytickalign         = xtickalign
					yticksize          = xticksize
					yticksmirrored     = false
					ytickwidt          = xtickwidt
					ytrimspine         = false

					Linewidth = 4
					xlabelSize = 30
					xticksize = 10
					xgridvisible = false
					Width = 800 # 800
					Height = 200


200

In [21]:

using GLMakie
GLMakie.activate!
Makie.inline!(false)  # Make sure to inline plots into Documenter output!


function HEATMAP_TIME(;Path=Path, NameOutput="q_land", Layer=1)
	Output_NCDatasets = NCDatasets.NCDataset(Path)

	Data = Output_NCDatasets[NameOutput]

	Data = Array(Data)

	Dimensions = length(size(Data))

	if Dimensions == 3
		N_Lon = size(Data)[1]
		N_Lat  = size(Data)[2]
		N_Time  = size(Data)[3]

	elseif Dimensions == 4
		N_Lon = size(Data)[1]
		N_Lat  = size(Data)[2]
		N_Time  = size(Data)[4]
	end

	Pmin, Pmax = extrema(x for x ∈ skipmissing(Data) if !isnan(x))
	@show Pmin Pmax

	function DATA_3D_2_2D(Data; iTime=iTime, Dimensions=Dimensions, Layer=Layer)
		if Dimensions == 4
			return Data[:,:, Layer, iTime]
		elseif Dimensions == 3
			return Data[:,:, iTime]
		end
	end


	Fig = Figure(size=(Width, Height * 4.0))

	Ax_1 = Axis(Fig[1, 1], title=NameOutput, xlabelsize=xlabelSize, ylabelsize=xlabelSize, xticksize=xticksize, xgridvisible=xgridvisible, ygridvisible=xgridvisible)

	sg = SliderGrid(Fig[2, 1],
	(label="iTime", range=1:1:N_Time, startvalue=1),
	width=550, tellheight=true)

	iTime = sg.sliders[1].value

	Data_Time = lift((iTime) -> DATA_3D_2_2D(Data; iTime=iTime, Dimensions), iTime)


	Data_Plot = heatmap!(Ax_1, 1:N_Lon, 1:N_Lat, Data_Time, colorrange=(Pmin, Pmax), colormap =:hawaii50)

	Colorbar(Fig[1, 2], Data_Plot; label=NameOutput, width=20, ticks = Pmin:(Pmax-Pmin)/5:Pmax)

	Fig
end


HEATMAP_TIME (generic function with 1 method)

In [ ]:
using GLMakie
GLMakie.activate!
Makie.inline!(false)  # Make sure to inline plots into Documenter output!

HEATMAP_TIME(;Path=Path_NetCDFmeteo_Output, NameOutput="precip", Layer=1)


Pmin = 0.0
Pmax = 3.6
